<h1 align="center">Coursera IBM Capstone Project - Applied Data Science</h1>

 <h2 align="center">Prepared by Stephan G. Maher</h2>

<h3 align="center">Part 1 WEEK3 Final Assignment</h3>

<h3>I''m providing these markdown notes, but there is also comments for most cells </h3>

In [ ]:
#import packages for scrape,Geo Data and set pandas options
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import lxml
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import geocoder
#Import libraries for Clustering
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#Import geocoder to get location of Toronto
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#Import Folium to create map.
import folium

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

#### 1. Scrape Webiste and prepare initial Data Frame

In [ ]:
# specify web page  to be scraped, parse the HTML and reformat
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
# print(soup.prettify) #uncomment to see raw data

In [ ]:
# find the title of table
soup.title
soup.title.string

In [ ]:
#Pull wikitable sortable into a variable
my_table=soup.find('table', class_='wikitable sortable')
#my_table # un-comment to see raw data

In [ ]:
#Create lists of column data
A=[]
B=[]
C=[]
for row in my_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))  

In [ ]:
#Adjust lists to remove /n
a=[a[:-1] for a in A]
b=[b[:-1] for b in B]
c=[c[:-1] for c in C]

In [ ]:
# Create the base dataframe
# after this step perform data cleaning for df
df=pd.DataFrame(a,columns=['Postal Code'])
df['Borough']=b
df['Neighborhood']=c
print(df.shape)
print(df.head())

<h3>Cleaning df Steps</h3>
<h3>1 discard Not assigned Boroughs </h3>
<h3>2 duplicate postal codes with different neighborhoods should be combined into 1 postal code and a comma seperated list of Neighborhoods </h3>
<h3>3 If the neigborhood is Not Assigned, use the Borough Assignment </h3>
<h3>4 Create the shape of the final df </h3>
<h3>5 Submit workbook to Github </h3>
<h3> </h3>



In [ ]:
# 1 discard empty Borough rows
print(df.shape) #(180,3)
print(df['Borough'].value_counts()) #77 Not Assigned
df['Borough'].replace('Not assigned',np.nan,inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)
df['Postal Code'].sort_values()
df.reset_index(drop=True, inplace=True)
#print(df.head(10))
#df.shape #S/b 180-77 reindex shape(103,3)

<h3>A review of the data shows that there are 77 Boroughs Not Assigned </h3>
<h3>The initial shape (180,3> is adjusted by removing those 77 Boroughs </h3>
<h3>The final shape is (103,3) as displayed below along with the dataframe </h3>
<h3> a review of data shows no other cleaning steps are needed. </h3>
<h3> </h3>

<h3 align="center"> Submit Week3Final Question 1</h3>

In [ ]:
print(df.shape)
print(df.head())

<h2 align="center">Start Question 2 </h2>
<h3>1 create the function get_Geocode that will pull 'latitude' and 'longitude' for the submitted 'Postal Code' </h3>
<h3>2 create a function call that reads the df dataframe and submits each postal code and stores the codes and latitude, longitude info to series. </h3>
<h3>3 insert the series information to a dataframe df_geo </h3>
<h3> </h3>

In [ ]:
#Function to pull Geo Data by Postal Code(PC)

def get_Geocode(PC):
    lat_lng_coords = None
    i=0
    while (lat_lng_coords is None):
        g=geocoder.arcgis('{},Toronto, Ontario'.format(PC))
        lat_lng_coords = g.latlng
        i+=1
        if i > 70:
            return None
        else:      
            return lat_lng_coords
    


In [ ]:
#Loop through df Postal Codes and collect Lat long data
A=[]
B=[]
C=[]
for PC in df['Postal Code']:
    LL=get_Geocode(PC)
    if LL is None:
        print("Error")
        break
    else:
        A.append(PC)
        B.append(LL[0])
        C.append(LL[1])


In [ ]:
df_geo=pd.DataFrame(A,columns=['Postal Code'])
df_geo['Latitude']=B
df_geo['Longitude']=C
print(df_geo.shape)
print(df_geo.head(10))

<h3>The steps above run the geo-coder and pull the Latitude and longitude for each zip code</h3>
<h3>There aren't seperate Latitude and longitude for each neighborhood, but there are for each borough </h3>
<h3>I plan to use the Boroughs in Toronto City proper and rename them to neighborhoods </h3>
<h3>This leads to 4 boroughs which I will use as centriods </h3>

<h3>1 Sort the Original Dataframe and the Geo dataframe by Postal Code </h3>
<h3>2 merge the dataframes by matching Postal Codes </h3>
<h3>3 Print Question2 Dataframe head</h3>

In [ ]:
#sort the dataframes by postal code
df.sort_values(['Postal Code'])
df.reset_index(drop=True, inplace=True)
df_geo.sort_values(['Postal Code'])
df_geo.reset_index(drop=True, inplace=True)


<h3>Show dataframe needed for Question2 </h3>

In [ ]:
#Code to join the geographical columns from csv to project df by "Postal Code"
col = ['Postal Code']
df=df.join(df_geo.set_index(col), on=col)
print(df.shape)
print(df.head())

<h2 align="center"> Submit Week3Final Question 2</h2>
<h2 align="center">See Shape and head above</h2>

<h2 align="center">Question 3 Analyse Toronto neighborhood</h2>

<h3>I'm selecting the Boroughs inside the city limits of Toronto </h3>
<h3>I then create the datafeame 'Toronto_Data' which exlcudes the neigborhoods outside of Toronto </h3>
<h3>Finally I rename the 'Borough' column to 'neighborhood' </h3>
<h3>Individual Neigborhoods would have the same(duplicate) postal code </h3>
<h3>Each Borough has a unique Postal Code</h3

In [ ]:
#Select Boroughs in Toronto and change the description to neighborhood
Toronto_data = df[['Postal Code','Borough','Latitude','Longitude']]
Toronto_data = Toronto_data[Toronto_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data = Toronto_data.rename(columns={'Borough':'neighborhood'})
print(Toronto_data.head())
print(Toronto_data.shape)
uniqueValues = Toronto_data['neighborhood'].unique()
print(uniqueValues)

In [ ]:
#get city coordinates for map background
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#Create the map and plot all current data points by Borough in the City
Toranto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#Plot the neighbourhoods
for lat, lng, postal_code, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['neighborhood'], Toronto_data['Postal Code']):
    label = '{}, {}'.format(postal_code, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toranto_map)  
Toranto_map

In [ ]:
# @hidden_cell
CLIENT_ID = 'QXBRRBOS22KLNITA013HBQPZ0ZQ5EPT3QQY4P5BVWF3EGJPS' # your Foursquare ID
CLIENT_SECRET = '5OXNVM4M2C11UDXGAKVNCHNY5GXCPRAXMMYDC1VUOKBL0RWQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

<h3> This section will get venue information near each data point so we can use this data to cluster</h3>

In [ ]:
#Get venues within 500 meters of Neighborhood locations
LIMIT = 100 # limit Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
#url # display URL

In [ ]:
#Get the result Data
results = requests.get(url).json()
#results #uncomment this line to see raw JSON.

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#Copy Category function
# function that extracts the category of the venue
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#Get and normalize venues
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
#Call Function and run against our neighborhood list
Toronto_venues = getNearbyVenues(names=Toronto_data['neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

In [ ]:
#Print information about neigborhood venues
print(Toronto_venues.shape)
print(Toronto_venues.head())
print(Toronto_venues.groupby('Neighborhood').count())
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))


In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
print(Toronto_onehot.shape)
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
#Toronto_grouped

In [ ]:
#Get top 5 venues per Neighborhood
num_top_venues = 5
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(4)

<h3>Above is the dataframe of the venues that we will use for  K-means,/h3>
<h3> </h3>
<h2 align="center">Starting Clustering </h2>

<h3>1 Set the number of centroids and kmeans algorythm </h3>
<h3>2 Add the clustering labels = venues by neighborhood </h3>
<h3>3 Create a map of the data </h3>

In [ ]:
# set number of clusters
kmc = 4

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kmc, random_state=6).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

Toronto_merged.head() # check the last columns!

In [ ]:
      # create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kmc)
ys = [i + x + (i*x)**2 for i in range(kmc)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

<h3>The final four frames show the Most common locations for each cluster</h3>

In [ ]:
print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[0] + list(range(5, Toronto_merged.shape[1]))]])


In [ ]:
print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]])


In [ ]:
print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]])


In [ ]:
print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[3] + list(range(5, Toronto_merged.shape[1]))]])

<h2 align="center">End of Notebook: Be kind when grading</h2>